In [122]:
import pandas as pd
from shared_functions import * 
import os

In [123]:
pd.set_option("display.max_columns", 25)
pd.set_option("display.max_colwidth",20)

In [124]:
notebook = os.path.basename(globals()['__vsc_ipynb_file__'])

Open File

In [125]:
source_path = f"{raw_files_folder}/geoplaces2.csv"

In [126]:
df_orig = read_files(source_path, notebook)  

Validate column names in the source

In [127]:
source_columns = df_orig.columns.to_list()


In [128]:
expected_columns = ['placeID','latitude','longitude','the_geom_meter','name','address','city','state','country','fax',
 'zip','alcohol','smoking_area','dress_code','accessibility','price','url','Rambience','franchise','area','other_services']


In [129]:
source_columns_name_check(source_columns ,expected_columns, notebook)

Remove leading, trailing whitespaces

In [130]:
df = strip_df(df_orig)

Remove rows where mandatory data are missing 


In [131]:
# column is empty

df_empty_col = df.drop("fax", axis=1)

In [132]:
# columns that are not allowed to have missing values

mandatory_columns_geoplaces = ["placeID" , "name", "address", "city", "state", "country"]


In [133]:
df_mandatory_cols = mandatory_columns_null_handling(df_empty_col,mandatory_columns_geoplaces, notebook)


Handle missing data in optional columns 

In [134]:
df_optional_cols = optional_columns_null_handling(df_mandatory_cols, mandatory_columns_geoplaces, notebook)


Replace specified chars, capitalise first letter

In [135]:
format_columns_geoplaces = ["name", "address", "city", "state", "country", "zip", "alcohol", "smoking_area", "dress_code", "accessibility", "price", "Rambience", "franchise", "area", "other_services"]


In [136]:
replace_char_dict_geoplaces = {"_": " ", "-": " "}

In [137]:
df_format_cols = format_columns(df_optional_cols, format_columns_geoplaces, replace_char_dict_geoplaces, notebook)


In [138]:
df_format_cols[["latitude", "longitude"]] = df_format_cols[["latitude", "longitude"]].astype(float)

Abbrevation expansion

In [139]:
df_abbr = df_format_cols.copy(deep=True)
df_abbr.loc[:, "city": "state"] = df_format_cols.loc[:, "city": "state"].apply(lambda x: x.replace({'S.L.P.':'San Luis Potosi', 'Slp':'San Luis Potosi', r"\bVictoriab":'Ciudad Victoria', 'Cd Victoria':'Ciudad Victoria', 'Cd. Victoria':'Ciudad Victoria'}
                                    , regex=True))


In [140]:
df_abbr["franchise"].replace({"F": "False", "T": "True"}, inplace=True)

Constraints

In [141]:
def constraints():
    try: 
        if df_abbr["franchise"].isin(["False", "True"]).all():
            pass
        else:
            raise ValueError
    
    except Exception:
        trace = traceback.format_exc(limit=1)

        with open("log.txt", "a") as f:
            print(f"An error occured running the <{notebook}>", file=f)
            print()
            print(trace, file=f)  

        sys.exit()

In [142]:
constraints()

Check duplicates

In [143]:
df_final_geoplaces = df_abbr.drop_duplicates()


In [144]:
geo_unique_place_id = df_final_geoplaces["placeID"].unique()

In [145]:
%store geo_unique_place_id

Stored 'geo_unique_place_id' (ndarray)
